Here we present an example to **recover smoothed labels from untrained LeNet on cifar10**.

Due to time limit, instead of 1000 examples, here we experiment 100 times with 10 samples for every class, and got **100%** accuracy.

In [2]:
import torch
import random
from typing import OrderedDict
from exp import cross_entropy_for_onehot
from recovering import label_recovery
from tqdm import tqdm
import numpy as np
from datetime import datetime
import time
seed=2023
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.enabled = False
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
CONFIG=OrderedDict(device=torch.device('cpu'),
    dataset="cifar10",
    network="lenet",
    opt="lbfgs",
    type='label_smooth',
    pretrained=False,
    criterion=cross_entropy_for_onehot,
    lr=0.5,
    bound=100,
    iteration=200,
    initia=1.,
    coefficient=4)
test=label_recovery(CONFIG)
test.datadir='/home/yanbo.wang/'+test.datadir
datalist=np.load('additional_files/mixup_list_cifar10.npy')
# datalist=np.load('additional_files/mixup_list_imagenet.npy',allow_pickle=True)
# datalist=np.load('additional_files/dataset_cifar100.csv',allow_pickle=True)
exp=np.zeros((10,100,8))#index,prob,featureloss,real_scalar,reco_scalar,scalar_loss,success,time 
for i in tqdm(range(10)):
    choice_index=np.random.choice(datalist[i],10)
    for i_exp, ind in enumerate(choice_index):
        prob=random.uniform(0,0.5)
        #prob=0
        test.setup(ind,prob)
        exp[i,i_exp,0],exp[i,i_exp,1]=ind,prob
        start_time=time.time()
        exp[i,i_exp,6]=test.label_reco()
        if exp[i,i_exp,6]==-1:
            exp[i,i_exp,6]=test.pso()
        exp[i,i_exp,7]=time.time()-start_time
        exp[i,i_exp,3],exp[i,i_exp,4],exp[i,i_exp,5]=test.ground_truth,test.scalar,test.ground_truth-test.scalar
        if exp[i,i_exp,6] ==1 or exp[i,i_exp,6] == 0:
            exp[i,i_exp,2]=((test.recover_tensor-test.net.temp)**2).sum()

  0%|          | 0/10 [00:00<?, ?it/s]

flip!
epoch is 6
epoch is 3
flip!
epoch is 6
flip!
epoch is 6
flip!
epoch is 6
flip!
epoch is 6
flip!
epoch is 6
flip!
epoch is 6
flip!
epoch is 6


 10%|█         | 1/10 [00:04<00:37,  4.12s/it]

epoch is 3
flip!
epoch is 6
epoch is 3
flip!
epoch is 6
flip!
epoch is 6
flip!
epoch is 6
flip!
epoch is 6
epoch is 3
epoch is 3
flip!
epoch is 6


 20%|██        | 2/10 [00:07<00:31,  3.88s/it]

...

epoch is 3
flip!
epoch is 6
flip!
epoch is 6
flip!
epoch is 6
flip!
epoch is 6
flip!
epoch is 6
epoch is 3
epoch is 3
epoch is 3
flip!
epoch is 6


 90%|█████████ | 9/10 [00:36<00:04,  4.09s/it]

epoch is 3
epoch is 3
flip!
epoch is 6
flip!
epoch is 6
flip!
epoch is 6
epoch is 3
flip!
epoch is 6
flip!
epoch is 6
flip!
epoch is 6
flip!
epoch is 6


100%|██████████| 10/10 [00:40<00:00,  4.07s/it]

flip!
epoch is 6


Under such settings, gradient-based algorithm (L-BFGS) is able to find the global optimum.

Below is the tiny experiment for mixup. It takes about 18min to finish recover 135 examples.

When L-BFGS did not get the optimal scalar, PSO is called to search iteratively. Adjusting searching bound, seed population as well as every searching range will alter the running time and searching accuracy (and error). It is a time-accuracy trade-off. 

In [3]:
from itertools import combinations
seed=2023
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)
CONFIG=OrderedDict(device=torch.device('cpu'),
    dataset="cifar10",
    network="lenet",
    opt="lbfgs",
    type='mixup',
    pretrained=False,
    criterion=cross_entropy_for_onehot,
    lr=0.5,
    bound=100,
    iteration=200,
    initia=1.,
    coefficient=2)
test=label_recovery(CONFIG)
test.datadir='/home/yanbo.wang/'+test.datadir
mixup_list=np.load('additional_files/mixup_list_cifar10.npy')
exp=np.zeros((45,20,9))#index,prob,featureloss,real_scalar,reco_scalar,scalar_loss,success,time 
combination_list=list(combinations(range(10),2))
for i in tqdm(range(45)):
    item=combination_list[i]
    for ii in range(3):# it is ok to change the number of experiments for every combination. Currently we have 3*45=135 experiments in total.
        ind=[random.randint(0,999),random.randint(0,999)]
        prob=random.uniform(0,1)
        test.setup([mixup_list[item[0],ind[0]],mixup_list[item[1],ind[1]]],[1-prob,prob])
        exp[i,ii,0],exp[i,ii,1],exp[i,ii,2]=ind[0],ind[1],prob
        start_time=time.time()
        exp[i,ii,7]=test.label_reco()
        if exp[i,ii,7]==-1:
            exp[i,ii,7]=test.pso(50)
        exp[i,ii,8]=time.time()-start_time
        exp[i,ii,4],exp[i,ii,5],exp[i,ii,6]=test.ground_truth,test.scalar,test.ground_truth-test.scalar
        if exp[i,ii,7] ==1 or exp[i,ii,7] == 0:
            exp[i,ii,3]=((test.recover_tensor-test.net.temp)**2).sum()

  0%|          | 0/45 [00:00<?, ?it/s]

flip!
epoch is 6
flip!
epoch is 6


  2%|▏         | 1/45 [00:01<00:46,  1.06s/it]

epoch is 3
flip!
epoch is 6
flip!
epoch is 6


  4%|▍         | 2/45 [00:02<00:54,  1.26s/it]

...

[-2.03905833] [3.33778873e-11]
successfully find the ground_truth [-2.03905833]
flip!
epoch is 6
flip!
epoch is 6


 98%|█████████▊| 44/45 [18:09<00:53, 53.44s/it]

flip!
epoch is 6
epoch is 3
epoch is 3


100%|██████████| 45/45 [18:10<00:00, 24.23s/it]

flip!
epoch is 6


In [6]:
count=0
for i in exp[:,:,7].reshape((-1)):
    if i<0:
        count+=1

In [7]:
count

0